# Автоматизация отчетности по ленте новостей

## Описание проекта

Необходимо автоматизировать базовую отчетность приложения с помощью Airflow. Наладить автоматическую отправку аналитических сводок по ленте новостей в телеграм каждое утро. 

Отчет должен содержать:
 - текстовую информацию о значениях ключевых метрик за предыдущий день
 - график с значениями метрик за предыдущие 7 дней

В отчете должны быть отображены следующие метрики: 
 - DAU
 - Просмотры
 - Лайки
 - CTR

## Данные

Таблица feed_actions - данные по ленте новостей:
 - user_id - уникальный номер пользователя
 - post_id - уникальный номер поста
 - action - лайк или просмотр
 - time - время события
 - gender - пол пользователя
 - age - возраст пользователя
 - country - страна пользователя
 - city - город пользователя
 - os - операционная система устройства пользователя
 - sourse - источник привлечения пользователя (organic - поисковик, ads - реклама)
 - exp_group - номер экспериментальной группы

## Результат

Ежедневный отчет отправляется в 11 утра в телеграм. Отчет содержит текстовое сообщение с значениями заданных метрик за прошлый день и три графика значениями метрик за предыдущую неделю(DAU, CTR, Actions)

## Скрипт сборки отчета

In [ ]:
# coding=utf-8

import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import pandahouse as ph
from datetime import datetime, timedelta

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220820'
}

default_args = {
    'owner': 'e-varshavskaja-10',
    'depends_on_past': False,
    'retries': 2,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2022, 9, 11), 
}

schedule_interval = '0 11 * * *'

my_token = '**********:_______________________'
bot = telegram.Bot(token=my_token)
chat_id = **********

@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False)
def dag_evarshavskaya_bot():
    
    @task()
    def extract_report():
        q = '''SELECT 
                  toDate(time) as date,
                  COUNT(DISTINCT user_id) as users,
                  SUM(action='view') as views,
                  SUM(action='like') as likes,
                  likes/views as ctr
               FROM simulator_20220820.feed_actions
               WHERE toDate(time) = yesterday()
               GROUP BY toDate(time)
       '''
        report = ph.read_clickhouse(query=q, connection=connection)
        return report


    @task()
    def extract_actions():
        
        q = '''SELECT 
                    toDate(time) as date,
                    COUNT(DISTINCT user_id) as users,
                    SUM(action='view') as views,
                    SUM(action='like') as likes,
                    likes/views as ctr
                FROM simulator_20220820.feed_actions
                WHERE toDate(time) between (yesterday() - 6) and yesterday()
                GROUP BY toDate(time)
       '''
        tab = ph.read_clickhouse(query=q, connection=connection)
        return tab

   
    @task()
    def send_message(report):
        
        msg = 'Метрики за вчерашний день: DAU - {}, Просмотры - {}, Лайки - {}, CTR - {}% ' \
               .format(report['users'][0], report['views'][0], report['likes'][0], round((report['ctr'][0]*100), 1))
        bot.sendMessage(chat_id=chat_id, text=msg)
        
    @task()
    def send_dau(tab):
        
        sns.lineplot(x = tab['date'], y = tab['users'])
        plt.title('DAU')
        plt.ylabel('')
        plt.xticks(rotation=20, size=9)
        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.seek(0)
        plot_object.name = 'dau.png'
        plt.close()
        bot.sendPhoto(chat_id=chat_id, photo=plot_object)
        
    @task()
    def send_ctr(tab):
        
        sns.lineplot(x = tab['date'], y = tab['ctr'])
        plt.title('CTR')
        plt.ylabel('')
        plt.xticks(rotation=20, size=9)
        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.seek(0)
        plot_object.name = 'ctr.png'
        plt.close()
        bot.sendPhoto(chat_id=chat_id, photo=plot_object)
        
    @task()
    def send_actions(tab): 
        
        sns.lineplot(x = tab['date'], y = tab['views'])
        sns.lineplot(x = tab['date'], y = tab['likes'])
        plt.title('Просмотры и лайки')
        plt.ylabel('')
        plt.legend(['views', 'likes'])
        plt.xticks(rotation=20, size=9)
        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.seek(0)
        plot_object.name = 'actions.png'
        plt.close()
        bot.sendPhoto(chat_id=chat_id, photo=plot_object)
        
    report = extract_report()
    tab = extract_actions()
    send_message(report)
    send_dau(tab)
    send_ctr(tab)
    send_actions(tab)
    
    
dag_evarshavskaya_bot = dag_evarshavskaya_bot()